In [1]:
#!/usr/bin/env python3
"""
Akantu 正確材料 API 使用
基於發現的正確簽名: registerNewMaterial(material_name, material_type, option)
"""

# import akantu.py11_akantu as aka
import akantu as aka
# import numpy as np
import os

def test_correct_material_registration():
    """
    使用正確的材料註冊 API
    """
    
    print("TESTING CORRECT MATERIAL REGISTRATION API")
    print("="*50)
    
    # 創建簡單網格
    mesh_file = "correct_material_test.msh"
    gmsh_content = """$MeshFormat
2.2 0 8
$EndMeshFormat
$PhysicalNames
1
2 1 "triangle"
$EndPhysicalNames
$Nodes
3
1 0 0 0
2 1 0 0
3 0.5 1 0
$EndNodes
$Elements
1
1 2 2 1 1 1 2 3
$EndElements
"""
    
    with open(mesh_file, 'w') as f:
        f.write(gmsh_content)
    
    try:
        mesh = aka.Mesh(2)
        mesh.read(mesh_file)
        
        model = aka.SolidMechanicsModel(mesh)
        
        print("Registering material with correct API...")
        
        # 使用正確的 registerNewMaterial 簽名
        # registerNewMaterial(material_name: str, material_type: str, option: str = '')
        
        # 嘗試不同的材料類型
        material_types_to_try = [
            ("my_material", "elastic", ""),
            ("my_material", "linear_elastic", ""),  
            ("my_material", "isotropic_linear_elastic", ""),
            ("my_material", "Material", ""),
            ("my_material", "elastic_isotropic", ""),
            ("granite", "elastic", "plane_stress"),
            ("granite", "elastic", "plane_strain"),
            ("test", "elastic", "2d"),
        ]
        
        material_registered = False
        
        for mat_name, mat_type, option in material_types_to_try:
            try:
                print(f"  Trying: registerNewMaterial('{mat_name}', '{mat_type}', '{option}')")
                model.registerNewMaterial(mat_name, mat_type, option)
                print(f"✓ Successfully registered material: {mat_name}, {mat_type}, {option}")
                material_registered = True
                registered_name = mat_name
                break
            except Exception as e:
                print(f"    Failed: {str(e)[:80]}...")
                continue
        
        if not material_registered:
            print("✗ Could not register any material type")
            return False
        
        # 初始化模型
        model.initFull()
        nb_materials = model.getNbMaterials()
        print(f"✓ Model initialized with {nb_materials} materials")
        
        # 設置材料屬性
        if nb_materials > 0:
            try:
                # 嘗試通過索引獲取材料
                material = model.getMaterial(0)
                material.setParam("E", 1000.0)
                material.setParam("nu", 0.3)  
                material.setParam("rho", 1e-9)
                print("✓ Material properties set via index")
            except Exception as e:
                try:
                    # 嘗試通過名稱獲取材料
                    material = model.getMaterial(registered_name)
                    material.setParam("E", 1000.0)
                    material.setParam("nu", 0.3)
                    material.setParam("rho", 1e-9)
                    print("✓ Material properties set via name")
                except Exception as e2:
                    print(f"⚠ Material property setting failed: {e}, {e2}")
        
        # 設置邊界條件和求解
        displacement = model.getDisplacement()
        external_force = model.getExternalForce()
        
        displacement.fill(0.0)
        external_force.fill(0.0)
        
        # 固定第一個節點
        displacement[0, :] = 0.0
        
        # 在第三個節點施加載荷
        external_force[2, 1] = -10.0
        
        print("Solving with properly registered material...")
        model.solveStep()
        print("✓ SOLUTION SUCCESSFUL WITH CORRECT MATERIAL API!")
        
        # 結果
        disp_result = model.getDisplacement()
        max_disp = np.max(np.abs(disp_result))
        print(f"  Maximum displacement: {max_disp:.6f} mm")
        
        # 詳細結果
        nodes = mesh.getNodes()
        for i, (node, disp) in enumerate(zip(nodes, disp_result)):
            print(f"  Node {i}: pos=({node[0]:.2f}, {node[1]:.2f}), disp=({disp[0]:.6f}, {disp[1]:.6f})")
        
        return True
        
    except Exception as e:
        print(f"✗ Correct material API test failed: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    finally:
        if os.path.exists(mesh_file):
            os.remove(mesh_file)

def test_3d_with_correct_materials():
    """
    使用正確的材料 API 測試 3D 問題
    """
    
    print("\nTESTING 3D WITH CORRECT MATERIAL API")
    print("="*50)
    
    # 創建 3D 四面體網格
    mesh_file = "3d_correct_material.msh"
    gmsh_content = """$MeshFormat
2.2 0 8
$EndMeshFormat
$PhysicalNames
1
3 1 "tetrahedron"
$EndPhysicalNames
$Nodes
4
1 0 0 0
2 1 0 0
3 0.5 1 0
4 0.5 0.5 1
$EndNodes
$Elements
1
1 4 2 1 1 1 2 3 4
$EndElements
"""
    
    with open(mesh_file, 'w') as f:
        f.write(gmsh_content)
    
    try:
        mesh = aka.Mesh(3)
        mesh.read(mesh_file)
        
        model = aka.SolidMechanicsModel(mesh)
        
        # 註冊 3D 材料
        print("Registering 3D material...")
        
        material_3d_types = [
            ("granite_3d", "elastic", ""),
            ("granite_3d", "linear_elastic", ""),
            ("granite_3d", "elastic", "3d"),
        ]
        
        material_registered = False
        for mat_name, mat_type, option in material_3d_types:
            try:
                model.registerNewMaterial(mat_name, mat_type, option)
                print(f"✓ 3D Material registered: {mat_name}, {mat_type}, {option}")
                material_registered = True
                break
            except Exception as e:
                print(f"  3D material {mat_type} failed: {str(e)[:60]}...")
                continue
        
        if not material_registered:
            print("✗ Could not register 3D material")
            return False
        
        # 初始化和設置材料屬性
        model.initFull()
        
        material = model.getMaterial(0)
        material.setParam("E", 30000.0)    # 花崗岩彈性模量
        material.setParam("nu", 0.25)      # 泊松比
        material.setParam("rho", 2.65e-9)  # 密度
        
        print("✓ 3D Material properties set")
        
        # 3D 邊界條件
        displacement = model.getDisplacement()
        external_force = model.getExternalForce()
        nodes = mesh.getNodes()
        
        displacement.fill(0.0)
        external_force.fill(0.0)
        
        # 固定底部節點 (z=0)
        for i, node in enumerate(nodes):
            if abs(node[2]) < 1e-6:
                displacement[i, :] = 0.0
                print(f"  Fixed node {i}: {node}")
        
        # 在頂部節點施加載荷 (z=1)
        for i, node in enumerate(nodes):
            if abs(node[2] - 1.0) < 1e-6:
                external_force[i, 2] = -100.0  # 向下載荷
                print(f"  Loaded node {i}: {node}")
        
        # 求解 3D 問題
        print("Solving 3D problem with materials...")
        model.solveStep()
        print("✓ 3D SOLUTION SUCCESSFUL!")
        
        # 結果
        disp_result = model.getDisplacement()
        max_disp = np.max(np.abs(disp_result))
        print(f"  3D Maximum displacement: {max_disp:.6f} mm")
        
        for i, (node, disp) in enumerate(zip(nodes, disp_result)):
            disp_mag = np.linalg.norm(disp)
            print(f"  Node {i}: pos=({node[0]:.2f}, {node[1]:.2f}, {node[2]:.2f}), |disp|={disp_mag:.6f}")
        
        # 檢查應力
        try:
            stress = model.getStress(aka.ElementType._tetrahedron_4)
            max_stress = np.max(np.abs(stress))
            print(f"  Maximum stress: {max_stress:.2f} MPa")
        except Exception as e:
            print(f"  Stress calculation: {e}")
        
        return True
        
    except Exception as e:
        print(f"✗ 3D test failed: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    finally:
        if os.path.exists(mesh_file):
            os.remove(mesh_file)

def main():
    """
    使用發現的正確 API 進行測試
    """
    
    print("AKANTU CORRECT API TESTING")
    print("="*70)
    print(f"Using Akantu version: {aka.getVersion()}")
    
    # 測試 1: 2D 正確材料 API
    success_2d = test_correct_material_registration()
    
    if success_2d:
        print("\n🎉 2D WITH MATERIALS WORKS!")
        
        # 測試 2: 3D 正確材料 API  
        success_3d = test_3d_with_correct_materials()
        
        if success_3d:
            print("\n🚀 BREAKTHROUGH! 3D WITH MATERIALS WORKS!")
            print("="*70)
            print("READY TO BUILD COMPLETE ABAQUS REPLICATION!")
            print("Key discoveries:")
            print("  ✓ Material registration: registerNewMaterial(name, type, option)")
            print("  ✓ 2D materials working")
            print("  ✓ 3D materials working") 
            print("  ✓ Boundary conditions working")
            print("  ✓ Static analysis working")
            print("  ✓ Results extraction working")
            print("\nNext: Build multi-component block assembly!")
        else:
            print("\n✅ At least 2D is working - can build from there")
    else:
        print("\n❌ Still fundamental issues")

if __name__ == "__main__":
    main()

AKANTU CORRECT API TESTING


AttributeError: module 'akantu' has no attribute 'getVersion'